In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
torch.set_grad_enabled(True)
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [2]:
train_set = torchvision.datasets.FashionMNIST(
            root='./data/FashionMNIST',
            train=True,
            download=True,
            transform=transforms.Compose([
                transforms.ToTensor()
            ]))

In [3]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)

In [3]:
for b in train_loader:
    im,bat=b

NameError: name 'train_loader' is not defined

In [ ]:
im=im.to('cuda:0')

In [ ]:
im.device

In [ ]:
type(im)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

torch.set_printoptions(linewidth=120)

In [ ]:
len(train_set)

In [ ]:
train_set.targets

In [ ]:
train_set.targets.bincount()

In [ ]:
sample= next(iter(train_set))

In [ ]:
len(sample)

In [ ]:
type(sample)

In [4]:
image,label=sample

NameError: name 'sample' is not defined

In [5]:
image.shape

NameError: name 'image' is not defined

In [6]:
label

NameError: name 'label' is not defined

In [7]:
plt.imshow(image.squeeze(), cmap='gray')
print('label:',label)

NameError: name 'plt' is not defined

In [8]:
import torch.nn as nn

In [9]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        # (1) input layer
        t = t
        
        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)
        
        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)
        
        return t

In [10]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [11]:
type(image)

NameError: name 'image' is not defined

In [15]:
network = Network().cuda()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100,num_workers=1)
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(100):
    
    total_loss = 0
    total_correct = 0
    
    for batch in train_loader: # Get Batch
        images, labels = batch 
        images=images.to('cuda:0')
        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/aditya/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/aditya/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/home/aditya/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 79, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/aditya/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 79, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/aditya/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 70, in default_collate
    return torch.tensor(batch)
  File "/home/aditya/anaconda3/lib/python3.7/site-packages/torch/c

In [122]:
from torch.utils.tensorboard import SummaryWriter

/home/aditya/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aditya/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aditya/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aditya/anaconda3/lib/python3.7/site-packages/tensorboar

In [130]:
network = Network().cuda()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

images, labels=next(iter(train_loader))
images=images.to('cuda:0')
grid=torchvision.utils.make_grid(images)

tb = SummaryWriter()
#tb.add_images('images', grid)
tb.add_graph(network, images)

for epoch in range(10):
    
    total_loss = 0
    total_correct = 0
    
    for batch in train_loader: # Get Batch
        images, labels = batch 
        images=images.to('cuda:0')
        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)
        
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number correct', total_correct,epoch)
    tb.add_scalar('Accuracy', total_correct/len(train_set),epoch)
    
    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight',network.conv1.weight,epoch)
    tb.add_histogram('conv1 weight grad', network.conv1.weight.grad,epoch)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )
tb.close()    

epoch 0 total_correct: 46796 loss: 350.92957097291946
epoch 1 total_correct: 51168 loss: 238.7834029197693
epoch 2 total_correct: 51957 loss: 217.41933929920197
epoch 3 total_correct: 52273 loss: 208.6096826195717
epoch 4 total_correct: 52421 loss: 205.6015720665455
epoch 5 total_correct: 52600 loss: 201.17500686645508
epoch 6 total_correct: 52788 loss: 196.42539279162884
epoch 7 total_correct: 52804 loss: 197.0631419569254
epoch 8 total_correct: 52953 loss: 192.57542376220226
epoch 9 total_correct: 52956 loss: 192.61861971020699
